In [ ]:
import boto3
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import lightgbm as lgb
from io import StringIO
from sklearn.metrics import mean_squared_error,roc_auc_score,precision_score

from cancellation_model import CancellationModel, load_training_data, load_prediction_data


In [ ]:
train = load_training_data()
test = load_prediction_data()

In [ ]:
params={}
params['learning_rate']=0.03
params['boosting_type']='gbdt' #GradientBoostingDecisionTree
params['objective']='binary' #Binary target feature
params['metric']='binary_logloss' #metric for binary classification
params['max_depth']=10
params['thres'] = 0.8
model = CancellationModel({"model_type": "lgb", "train_params" : params, "epocs" : 10})

In [ ]:
model.train(train)


In [ ]:
train_data, validation_data = train_test_split(
            train, test_size=0.3, random_state=1)
model.preprocess(train_data)

In [ ]:
model.predict(train_data, _preprocess = False)


In [ ]:
X_test = pd.DataFrame()
X_test['pred'], X_test['cancelled'] = model.predict(test)

In [ ]:
X_test.pred.sum()/X_test.cancelled.sum()


In [ ]:

print(X_test['pred'])
model_saved = "model.pkl"
model.dump(model_saved)
loaded_model = CancellationModel.load(model_saved)
X_test['pred'], X_test['cancelled'] = loaded_model.predict(test)
print(X_test['pred'])


In [ ]:
model.score(X_test['cancelled'], X_test['pred'])

In [ ]:
lgb.plot_importance(model.model, ignore_zero = False, importance_type='gain')

In [ ]:
train.columns


In [ ]:
lgb.plot_importance(model.model, ignore_zero = False, importance_type='split')